# BBLF AI Selector: Part 1: Pre Tournament Optimal Squad - Optimser

# 0. Prerequistes

In [7]:
# 0. Prerequistes

import pandas as pd
import numpy as np
import os
import random
from mip import Model, xsum, maximize, BINARY 

os.getcwd()
directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI'

# 1. Data Extraction 

In [ ]:
# 1. Data Extraction 
    # Optimal Number of Rounds
opt_round = 3

    # Pull in player price data csv file 

price_df = pd.read_csv(os.path.join(directory,'data/python_datasets/player_price.csv'), low_memory=False)
print(price_df)

# Player Role Flags

price_df["Wk_f"] = np.where((price_df["Role"] == "WK"), 1, 0)
price_df["Bat_f"] = np.where((price_df["Role"] == "WK") |(price_df["Role"] == "BAT") | (price_df["Role"] == "ALLR") , 1, 0)
price_df["Bowl_f"] = np.where((price_df["Role"] == "BOWL") | (price_df["Role"] == "ALLR") , 1, 0)
price_df = price_df[["Full_Name", "Price", "Team","Wk_f", "Bat_f", "Bowl_f", "Role", "Available"]].rename(columns = {"Full_Name": "Name"}) 
    # Pull in team fixture csv file

#team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_fixture.csv'), low_memory=False)
#team_fix_df = team_fix_df.melt(id_vars = "Team")
#team_fix_df = team_fix_df.rename(columns = {"variable": "Round", "value": "Opposition"})
#team_fix_df[["Round", "Game"]] = team_fix_df["Round"].str.split("_", expand = True)
#team_fix_df["Round"] = team_fix_df["Round"].str[1:].astype(int)
#team_fix_df = team_fix_df[["Team", "Round", "Opposition"]]
#team_fix_df = team_fix_df[team_fix_df.Round <= opt_round].dropna()
#print(team_fix_df)

team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_loc_fixture.csv'), low_memory=False)
team_fix_df = team_fix_df[team_fix_df.Round <= opt_round].dropna()
print(team_fix_df)

    # Pull player expected points csv file
exp_pts_df = pd.read_csv(os.path.join(directory,'data/python_datasets/bbl14_fullteam_model_score.csv'), low_memory=False).drop(["Unnamed: 0", "player"], axis = 1)
print(exp_pts_df)

    # Join team fixture to player price to create a row for each game
game_df = pd.merge(price_df , team_fix_df, left_on = ["Team"], right_on = ["Team"], how = "left")
print(game_df)

    # Join on expected points for each game
print(game_df.columns)
print(exp_pts_df.columns)
player_df_raw = pd.merge(game_df, exp_pts_df, left_on = ["Name", "Team", "Opposition", "Venue", "Home_f"], right_on = ["Name", "Team", "opp", "venue", "Home_f"], how = "left")
#player_df_raw["exp_pts"] = np.random.randint(0, 125, player_df_raw.shape[0])
player_df_raw["weight"] = 1
print(player_df_raw)

    # Aggregate by player name (calculate total expected points for each player for x number of rounds)
player_df = player_df_raw.groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available"], as_index=False).agg(
exp_pts=('exp_pts',"sum"))

      Last_Name First_Name First_Name_Short        Full_Name         player  \
0        Abbott       Sean               SA      Sean Abbott      SA Abbott   
1          Agar     Ashton               AC      Ashton Agar        AC Agar   
2          Agar        Wes               WA         Wes Agar        WA Agar   
3         Allen       Finn              NaN       Finn Allen     Finn Allen   
4         Allen     Fabian              NaN     Fabian Allen   Fabian Allen   
..          ...        ...              ...              ...            ...   
142       Wells        Jon               JW        Jon Wells       JW Wells   
143  Wildermuth       Jack               JD  Jack Wildermuth  JD Wildermuth   
144        Wood       Jack               JP        Jack Wood        JP Wood   
145      Wright        Mac                M       Mac Wright       M Wright   
146       Zampa       Adam                A       Adam Zampa        A Zampa   

      Price                 Team  Role  Available  

# 2. Optimisation Process (Round 1)

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 12
2. Atleast 1 wicketkeeper
3. Atleast 6 batters
4. Atleast 5 bowlers
5. Total budget of team is less than $1,802,250

In [9]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 12, range(len(price))
wk_cnt, total_wk = 1, range(len(price))
bat_cnt, total_bat = 6, range(len(price))
bowl_cnt, total_bowl = 5, range(len(price))
budget, total_budget = 1802250, range(len(price))

m = Model("knapsack")


x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]

print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))

# sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_pre_mistake.csv')


[<mip.entities.Var object at 0x00000228C44D7A90>, <mip.entities.Var object at 0x00000228C44D7A60>, <mip.entities.Var object at 0x00000228C44D7640>, <mip.entities.Var object at 0x00000228C44D7610>, <mip.entities.Var object at 0x00000228C44D75E0>, <mip.entities.Var object at 0x00000228C44D75B0>, <mip.entities.Var object at 0x00000228C44D7580>, <mip.entities.Var object at 0x00000228C44D74F0>, <mip.entities.Var object at 0x00000228C44D74C0>, <mip.entities.Var object at 0x00000228C44D7490>, <mip.entities.Var object at 0x00000228C44D7400>, <mip.entities.Var object at 0x00000228C44D7760>, <mip.entities.Var object at 0x00000228C44D77F0>, <mip.entities.Var object at 0x00000228C44D78E0>, <mip.entities.Var object at 0x00000228C44D7850>, <mip.entities.Var object at 0x00000228C44D79A0>, <mip.entities.Var object at 0x00000228C44D7940>, <mip.entities.Var object at 0x00000228C44D7820>, <mip.entities.Var object at 0x00000228C4498DC0>, <mip.entities.Var object at 0x00000228C44998D0>, <mip.entities.Var o